# PH213 Project Demonstration: Toast Falling from Counter
## Basic model (no iterations)
## 9/10/20


## The Problem

The basic problem is this. When toast teeters off the edge of a standard counter it seems to, remarkably consistently, land 'jelly side' down. I have read that this is no accident and that in fact the solution to the problem is to push the toast when you notice it going off the edge rather than try to stop it. I have done some loose experiments and find this to be empirically true. You should try it yourself and you can use a book if you don't want to waste a piece of toast. Now -- can I use basic dynamics and an numerical approach to explore this problem without getting caught up the analytic solution to the problem documented in various places in AJP. 

### Dependencies

These are the baseline dependencies that we have used in past notebooks. 

Here are the reference sites for [`matplotlib`](https://matplotlib.org/) and [`numpy`](http://www.numpy.org/)

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Defining constants

In this problem it seems prudent to allow for a variety of 'toast' like objects of different materials and sizes. To that end I want to establish a set of constants that describe the particular setting I am considering. Note that I am working in cm and s for convenience which may turn out to be a bad idea.

tlength (parallel to table edge - cm)   
twidth  (perpendicular to table edge - cm)   
tthick (yup - cm)   
tdensity (in case it matters - gr/cm<sup>3</sup>    
counterheight ( in cm)  
latspeed  ( cm/s)  
gravity  (cm/s<sup>2</sup>)   
anglimit (radians - generally set to $\pi/2$ but could be different)

## Rock Drop Calculation
Regardless of what else it's doing a physical object under the influence of gravity falls like a rock even if it's rotating as it does so. Think of this as squeezing the toast into a small ball of the same mass as rolling it off the edge.  

A quick calculation I should do as soon as these constants are established is calculate the time it take for the toast to fall from the counterheight to the floor. This is essentially a calculation we have done in lab and is given by some form of kinematic analysis and yields

$\Large time_{fall} = \sqrt{\frac{2*h}{g}}$

From a standard counter this should yield around 0.45 s. As I go along I check each number that results against a quick manual calculation to check moth units and magnitude.


## Problem Geometry

Here's a drawing with the labels that represent my current thinking.

<img src="images/toastBasic.png" alt="Geometric Diagram" title="Toast Sketch" />

In [12]:
tlength = 10.0
twidth = 10.0
tthick = 1.0
tdensity = 0.45
counterheight = 100.0
latspeed = 0.0
gravity = 981.0
anglimit = np.pi/2


floortime = np.sqrt([2.0*counterheight/gravity])
print("The time to fall to the floor from the counter is %.4f seconds" % (floortime))

The time to fall to the floor from the counter is 0.4515 seconds


## Physics Background
## Newton's 2nd Law (Rotational)

Something we often don't get to in PH211 is the rotational version of Newton's Laws. This will come up in ENGR 211 (Statics) and historically students make the transition pretty successfully. The core concept is that linear forces and rotational forces (torques) act independently to determine the behavior of an object. Point masses that we talked about most of the time in PH211 have no physical size so you can't rotate them. Toast on the other hand does have physical size and so we must consider both linear and rotational motion. The 30 second lecture about this is that every linear variable we discussed (x, v, and a and the components of each in the i,j, and k directions) has an angular equivalent. This table summarizes those equivalents:

x (position) -- $\theta$ (angular position), measured in radians.  
v (velocity) -- $\omega$ (angular velocity), which we ran into in circular motion and is measured in radians/s  
a (acceleration) -- $\alpha$ (angular acceleration), measured in (radians/s)/s  

You would not be surprised that all of our kinematic equations have exact equivalents for angular motion -- remember the kinematics equations are just descriptions of different plots and all I do is change the labels on the plot and we're done. So we get...  

$x(t) = x_0 + v*t$  
becomes   
$\theta (t) = \theta_0 + \omega * t$  
for constant linear or angular velocity.   


## Proof of Principle

My usual process is to try to code a 'simple' version of the problem before getting carried away. The intention is to keep it simple enough that I can manually check numbers to make sure I've got the units right. At the same time I try to establish whether the numbers I'm getting make sense based on experience or observation. Once I seem to be able to execute the calculations and basic structure seems right then I can try to improve the model one step at a time.

## The Setting

While the actual problem is what happens when toast slides of a counter a hopefully simple version of the problem is to imagine that the toast is support with 3/4 of it's width off the table and then released. When I do this in my house the toast seems to turn one complete revolution and land jelly side up. If I slide it slowly off the edge it seems to consistently land jelly side down.

In this case I want to start out by calculating the moment of inertia of the toast around it's center of mass in appropriate units. 

Toast is essentially just a book (rectangular box) so it's moment of intertia is   
  
$I_{toast} = \frac{1}{12}m a^2$  

where a is the side perpendicular to the axis of rotation. The mass is calculated from the density.

I added the calculation of the linear density of the toast (gr/cm of width) later when it seemed like this would simplify later calculations. This was not originally what I did.

In [13]:
lindensity = tdensity*tlength*tthick  # linear density of toast
tmass = lindensity*twidth
tinertiacm = tmass*(twidth*twidth)/12.0
print ("Inertia around CM: %.2f gr*cm^2" % (tinertiacm))
print ("Toast mass (gr): %.2f gr"% (tmass))

Inertia around CM: 375.00 gr*cm^2
Toast mass (gr): 45.00 gr


## Parallel Axis:

As the toast slides over the edge the axis of rotation changes. As a result the moment of interia changes as well. We calculate the moment of inertia around an axis parallel to the axis through the center of mass using the Parallel Axis Theorem (brilliant eh?).

I$_{\parallel} = $I$_{cm} + m d^2$ 

where m is the total mass of the object and d is the perpendicular distance between the desired axis and the parallel one through the center of mass.

Just to test the calculation....we expect the moment of inertia to go up as we move away from the center of mass!

In [4]:
dparallel = twidth/4.0 # this is just a temporary place holder to test the calculation....
tparallelI = tinertiacm + tmass*dparallel**2
print (" Inertia around parallel axis is %.2f gr*cm^2 at a distance of %.2f cm from cm" % (tparallelI, dparallel))

 Inertia around parallel axis is 656.25 gr*cm^2 at a distance of 2.50 cm from cm


## Torque

In broad strokes we now have the rotational equivalent of mass in Newton's 2nd Law. Now we need to determine the torque. The toast is always divided into two parts by the edge of the table. If we label the torque created by the overhanging part as positive (+) then the torque created by the portion over the table is (-). For this initial simplistic model I'm assuming the toast is 3/4 of the way off the table. The first term in parentheses is the distance to the center of mass of each part of the toast from the axis of rotation (the edge of the table). The second term in parentheses is the total mass of each section of the toast. The part NOT overhanging is asigned a negative value.

```python
torquepositive = (overhang*twidth/2)*(tmass*overhang*gravity)
torquenegative = -((1.0-overhang)*twidth/2)*(tmass*(1.0-overhang)*gravity)
totaltorque = torquepositive+torquenegative
```
At this time the numbers seem high but I'm not going to worry about it just yet. Manual checking verifies that they are appropriate.

In [5]:
overhang = 0.75
torquepositive = (overhang*twidth/2.0)*(tmass*overhang*gravity)
torquenegative = -((1.0-overhang)*twidth/2)*(tmass*(1.0-overhang)*gravity)
totaltorque = torquepositive+torquenegative
print ("positive torque: %.2f gr*cm^2/s^2; negative torque: %.2f gr*cm^2/s^2" % (torquepositive, torquenegative))
print ("total torque: %.2f gr*cm^2/s^2" % (totaltorque))

positive torque: 124157.81 gr*cm^2/s^2; negative torque: -13795.31 gr*cm^2/s^2
total torque: 110362.50 gr*cm^2/s^2


## Angular Acceleration

Find the angular acceleration $\alpha$:

$\huge \frac{\tau}{I} = \alpha$

Simple right?

In [6]:
angularaccel = totaltorque/tparallelI
print ("angular acceleration: %.2f 1/s^2" % (angularaccel))

angular acceleration: 168.17 1/s^2


## Initial Estimate:

Given that the toast can only rotate for $\pi/2$ radians before there is no torque we can estimate how long this acceleration could be applied.

$\theta _{final} = \theta_{initial} + \omega_{initial} t + 0.5 \alpha t^2 $

Once the time to turn $/pi/2$ radians is calculated then I can estimate the angular velocity of the toast as it comes off the edge. It will then rotate while it falls to the floor which will take about 0.45 s from a 1 m counter. Later I will need to make this an explict separate calculation. Here goes.....

In [7]:
angposinit = 0.0
angposfinal = np.pi/2
angvelinit = 0.0
timetohalfpi = np.sqrt([angposfinal*2/angularaccel])
print ("Estimated time to pi/2 rotation: %.4f " %(timetohalfpi))

Estimated time to pi/2 rotation: 0.1367 


In [8]:
angvelfinal = angvelinit + angularaccel*timetohalfpi
print ("Estimated angular velocity at pi/2 rotation: %.4f (rad/s)" %(angvelfinal))

Estimated angular velocity at pi/2 rotation: 22.9853 (rad/s)


In [9]:
floortime = 0.45
angposfloor = angvelfinal*floortime
print ("Estimated rotation at floor after fall time %.3f (s): %.4f (rad)" %(floortime,angposfloor))

Estimated rotation at floor after fall time 0.450 (s): 10.3434 (rad)


## Proof of Concept Analysis

Jelly side or not?

If the rotation when it gets to the floor is more than $\pi/2$ and less than $3 \pi/2$ then one would expect the toast to settle jelly side down. If it is within 5% of $\pi$ radians I would call it a perfect splat!

If it rotates more than $3 \pi/2$ then it becomes complicated. The fact that my test case gave me 10 radians (1 1/2 rotations) could give me pause but in thinking about it I have assumed that the angular acceleration is constant while the toast rotates which is clearly not the case. As the toast rotates the net torque is constantly dropping which is reducing the angular acceleration. This effect would lead to less rotation which seems satisfying.

## Next step....

Its always tempting to jump right to the actual problem and there are certainly people who can pull that off. My thesis advisor was someone who could write code like writing a paragraph and it nearly always worked. I'm not that smart so I have to sneak up on it. It seems to me the next helpful step is to sort out how to model the changing angular acceleration as the toast rotates.

### Continue on to Toast2.1....

